# Модули

## Какво е модул?

Всеки един Python файл (.py) на практика е модул. Освен това е възможно библиотека, написана на С, и вмъкната динамично също да бъде модул. Третия тип модули са вградените в езика такива.

В тази лекция се фокусираме върху първия тип и модули и как можем да ги създаваме, вмъкваме и боравим с тях.

## Как да създам модул?

Казахме, че всеки Python файл е валиден модул. 

Нека създадем един такъв с няколко дефиниции вътре (в папката с тази тетрадка вече би трябвало да се съдържа файл `hitchhikers.py`. Ако ***не се*** съдържа, изпълни следващатa клетка.)

In [5]:
# TODO

## `import`

Имената, функциите и класовете, които създадохме в този файл, не могат да бъдат достъпени директно от друг файл:

In [6]:
compute()

NameError: name 'compute' is not defined

Можем обаче да ги вмъкнем в друг файл (модул) чрез `import {името_на_модула}` (името на файла преди разширението `.py` се превръща в име на модула):

In [7]:
import hitchhikers

hitchhikers.compute()

Hm, I'll have to think about that. Return to this place in exactly 7.5 million years...


42

`import` освен, че интерпретира целия код на модула, добавя имената и дефинициите в един обект от тип модул, имащ името на модула. Затова и ги достъпваме чрез `името_на_модула.име_на_обекта`.

Какво се съдържа в един модул можем лесно да видим с `dir()`:

In [12]:
dir(hitchhikers)

['ANSWER',
 'TheGreatDeepThought',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'compute']

In [13]:
hitchhikers.ANSWER

42

In [15]:
computer = hitchhikers.TheGreatDeepThought()
computer.ask()

Shush! The show is back on.


In [16]:
hitchhikers.__name__

'hitchhikers'

In [11]:
dir()  # by default it shows the contents of the *current* module

['In',
 'Out',
 '_',
 '_1',
 '_10',
 '_2',
 '_7',
 '_8',
 '_9',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_exit_code',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'exit',
 'get_ipython',
 'hitchhikers',
 'module',
 'os',
 'quit',
 'sys']

In [17]:
__name__

'__main__'

## Добре, обаче `import` къде точно търси?

1. Директорията, в която се намира Python скрипта, който се изпълнява (или текущата, ако интерпретаторът е пуснат интерактивно)
2. Директориите, които са описани в `PYTHONPATH` променливата на средата
3. Лист от директории, зададен по време на инсталацията на Python

Този списък от възможни директории може да се види със `sys.path`:

In [18]:
import sys
sys.path

['/Users/alexander.ignatov/Documents/PythonCourse2022/13 - Modules',
 '/Users/alexander.ignatov/.vscode/extensions/ms-toolsai.jupyter-2022.9.1303220346/pythonFiles',
 '/Users/alexander.ignatov/.vscode/extensions/ms-toolsai.jupyter-2022.9.1303220346/pythonFiles/lib/python',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python310.zip',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload',
 '',
 '/Users/alexander.ignatov/Library/Python/3.10/lib/python/site-packages',
 '/opt/homebrew/lib/python3.10/site-packages',
 '/opt/homebrew/Cellar/pygments/2.13.0_1/libexec/lib/python3.10/site-packages']

## Варианти на `import`

С `from {module} import {something}, {something_else}, ...` можем да импортираме само определени имена от модула, като те биват добавени към съдържанието на текущия (т.е. достъпваме ги без името на оригиналния модул и точка отпред):

In [19]:
from hitchhikers import compute
compute()

Hm, I'll have to think about that. Return to this place in exactly 7.5 million years...


42

С `from {module} import {something} as {alias}, {something_else} as {other_alias}, ...` можем да прекръстим импортираните имена:

In [23]:
from hitchhikers import ANSWER, TheGreatDeepThought as Computer
comp = Computer()
comp.ask() == ANSWER

Shush! The show is back on.


False

In [27]:
"ANSWER" in dir()

True

Ако искаме абсолютно всички имена на вмъкнем и ползваме в текущия модул по този начин (без тези, започващи с подчертавка `_`), можем да използваме астериск `*`:

In [31]:
# изпълни тази клетка ако си изпълнил горните, за да се зачистят import-ите
del hitchhikers, ANSWER, TheGreatDeepThought, compute, Computer

NameError: name 'hitchhikers' is not defined

In [35]:
from hitchhikers import *

compute() == ANSWER

Hm, I'll have to think about that. Return to this place in exactly 7.5 million years...


True

Лимитация на астерикс синтаксиса е, че не може да използва в блок (може само на най-външното ниво на модула):

In [39]:
del compute, ANSWER, TheGreatDeepThought

NameError: name 'compute' is not defined

In [41]:
def obtain_answer():
    from hitchhikers import *
    return compute()

SyntaxError: import * only allowed at module level (3855291323.py, line 2)

Както казахме, по подразбиране from {module} import * вмъква абсолютно всички имена от `module`, които не започват с подчертавка. Имаме всъщност контрол над това, кое може да се вмъкне чрез астерикс, като дефинираме `__all__` във въпросния модул. Стойността му е лист от всички имена, които ще бъдат вмъкнати от `*`.

*Пример*:

След добавяне на
```python
__all__ = ['compute', 'TheGreatDeepThought']
```
в `hitchhikers.py`, следният код, изпълнен в `script.py` (в същата директория) ще хвърли `NameError`:
```python
from hitchhikers import *
print(hitchhikers.ANSWER)  # 💥
```

## Пакети

TODO